<h1 align="center">
<img src="https://drive.google.com/uc?export=view&id=1_ZkE3dnYg9KykzdZE6lIK2s_q_nbiKk_" width="160" height="160">
</h1>
<h1 align="center">Massive Data Analytics - 25577</h1>
<h3 align="center">Dr. Gholampour</h3>
<h5 align="center">Sharif University of Technology<br/>Electrical Engineering Department<br/>  Fall 2022</h5>

---

# Problem Set 02
<h4 align="center">Vahid Pourakbar     400205511</h4>

# Question 4

## Part 1

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("MDA_2022_PS2_Q4") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

22/12/23 15:27:43 WARN Utils: Your hostname, vahid-PC resolves to a loopback address: 127.0.1.1; using 172.27.210.240 instead (on interface wlp2s0)
22/12/23 15:27:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/23 15:27:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataframe = spark.read.csv("Sample_Traffic.csv", header=True, inferSchema=True)
dataframe.limit(5).show()

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
|        155|       81|       87625017|     87625017|               5|       161|2021-06-01 04:14:21|
|     631757|       81|        8652928|      8652928|               5|       161|2021-06-01 03:58:57|
|     631757|       81|        8548123|      8548123|               5|       161|2021-06-01 04:01:38|
|     631757|       81|       24715264|     24715264|               5|       161|2021-06-01 03:56:57|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+



In [4]:
# key = (plate, date), value=[list of device codes]
dataframe_rdd = dataframe["DEVICE_CODE","FINAL_CAR_KEY", "PASS_DAY_TIME"].rdd \
    .map(lambda x: ((x[1], x[2].date()), [x[0]])) \
        .reduceByKey(lambda x, y: x+y)
dataframe_rdd.take(10)

[((40682798, datetime.date(2021, 6, 1)), [206602]),
 ((101646406, datetime.date(2021, 6, 1)), [202901]),
 ((25742182, datetime.date(2021, 6, 1)), [128]),
 ((9305118, datetime.date(2021, 6, 1)),
  [230204, 900191, 900107, 100700820, 100700853, 900276]),
 ((58406846, datetime.date(2021, 6, 1)), [230204]),
 ((8361950, datetime.date(2021, 6, 1)),
  [631362,
   631757,
   631757,
   631362,
   631362,
   631757,
   900203,
   900203,
   100700820,
   900141,
   900269,
   900167,
   900226,
   900246]),
 ((8550078, datetime.date(2021, 6, 1)),
  [631829, 631361, 631759, 631360, 631361, 900222, 900202]),
 ((17261950, datetime.date(2021, 6, 1)),
  [205201,
   631795,
   137,
   900167,
   900240,
   900135,
   900171,
   22009977,
   900212,
   900142,
   900245,
   900265,
   100700804,
   631633,
   209103]),
 ((19783430, datetime.date(2021, 6, 1)), [205201]),
 ((25934494, datetime.date(2021, 6, 1)), [631830])]

# Apriori

In [126]:
support_threshold = 1000
frequent_cameras_count = dataframe_rdd.flatMap(lambda x: x[1]) \
    .map(lambda x: (x , 1)) \
        .reduceByKey(lambda x, y: x + y) \
            .filter(lambda x: x[1] >= support_threshold)
print("(path, traffic)")
frequent_cameras_count.take(10)

(path, traffic)


[(128, 8382),
 (631360, 3145),
 (900240, 15481),
 (631368, 15162),
 (100701144, 6176),
 (100700864, 29701),
 (100700824, 39176),
 (100700832, 4484),
 (144, 6307),
 (631832, 13483)]

In [127]:
len(frequent_cameras_count.collect())

398

In [128]:
frequent_cameras = frequent_cameras_count.map(lambda x: x[0])
frequent_cameras_list = frequent_cameras.collect()
frequent_cameras.take(5)

[128, 631360, 900240, 631368, 100701144]

In [112]:
pathes_rdd = dataframe_rdd.map(lambda x: x[1])
pathes_list = pathes_rdd.collect()
len(pathes_list)

1520404

## k = 2

In [129]:
# #  removing non-frequent pathes
# pathes_list_e1 = [[p for p in path if p in frequent_cameras_list] for path in pathes_list] 

# # removing one way pathes, duplicates and sorting 
# pathes_list_e2 = [sorted(list(set(i))) for i in pathes_list_e1 if len(set(i))>1]

# len(pathes_list_e2)

752531

In [134]:
pathes_rdd_e1 = pathes_rdd.map(lambda x: sorted(list(set(x)))) \
    .map(lambda x: [p for p in x if p in frequent_cameras_list]) \
        .filter(lambda x: len(x) > 1)     
pathes_rdd_e1.take(10)

[[230204, 900107, 900191, 900276, 100700820, 100700853],
 [631362, 631757, 900141, 900167, 900203, 900226, 900246, 900269, 100700820],
 [631360, 631361, 631759, 631829, 900202, 900222],
 [137,
  205201,
  209103,
  631633,
  631795,
  900135,
  900142,
  900167,
  900171,
  900212,
  900240,
  900245,
  900265,
  22009977,
  100700804],
 [113, 202101, 213402, 900185, 900243, 22010031],
 [153, 213402, 230103, 631368, 631763, 631765, 900164, 900276, 100700820],
 [213402, 631346, 900185, 22010031, 100700864, 100701144],
 [230101, 900185, 900265, 100700824],
 [202901, 203902, 900164, 900197, 900217, 100700832],
 [118,
  128,
  144,
  202601,
  205201,
  205202,
  206701,
  207101,
  208601,
  631763,
  631771]]

In [146]:
paired_frequent_cameras = pathes_rdd_e1.flatMap(lambda x: [(i,j) for i in x for j in x if i<j])
paired_frequent_cameras.take(10)

[(230204, 900107),
 (230204, 900191),
 (230204, 900276),
 (230204, 100700820),
 (230204, 100700853),
 (900107, 900191),
 (900107, 900276),
 (900107, 100700820),
 (900107, 100700853),
 (900191, 900276)]

In [147]:
paired_frequent_cameras_count = paired_frequent_cameras.map(lambda x: (x, 1)) \
    .reduceByKey(lambda x, y: x + y)
paired_frequent_cameras_count.take(10)

[((900107, 900191), 1535),
 ((900167, 900203), 490),
 ((900246, 100700820), 1812),
 ((631360, 900222), 281),
 ((631361, 631829), 1932),
 ((137, 900245), 9),
 ((205201, 900245), 27),
 ((209103, 631795), 330),
 ((209103, 900171), 242),
 ((631633, 900245), 173)]

In [148]:
len(paired_frequent_cameras_count.collect())

78886

In [149]:
frequent_camera_paires_count = paired_frequent_cameras_count.filter(lambda x: x[1]>= support_threshold)
frequent_camera_paires_count.take(10)

[((900107, 900191), 1535),
 ((900246, 100700820), 1812),
 ((631361, 631829), 1932),
 ((900142, 900212), 24495),
 ((900245, 22009977), 2578),
 ((202601, 900101), 1595),
 ((900215, 22009971), 2889),
 ((900222, 900228), 2357),
 ((900216, 100700862), 1129),
 ((900240, 100700862), 1615)]

In [150]:
frequent_camera_paires = frequent_camera_paires_count.map(lambda x: x[0])
frequent_camera_paires_list = frequent_camera_paires.collect()
len(frequent_camera_paires_list)

1545

## k = 3

In [204]:
def subset(x, d):
    if d == 4:
        return [(i, j, k, m) for i in x for j in x for k in x for m in x if i<j and j<k and k<m]
    elif d == 3:
        return [(i, j, k) for i in x for j in x for k in x if i<j and j<k]
    elif d == 2:
        return [(i, j) for i in x for j in x if i<j]

def flatten(list):
    return [item for sublist in list for item in sublist]

In [196]:
pathes_rdd_e2 = pathes_rdd_e1.map(lambda x: sorted(set(flatten([ss for ss in subset(x, 2) if ss in frequent_camera_paires_list])))) \
        .filter(lambda x: len(x) > 2)     
pathes_rdd_e2.take(10)

[[230204, 900107, 900191, 900276, 100700820, 100700853],
 [631362, 631757, 900167, 900203, 900226, 900246, 900269, 100700820],
 [631361, 631829, 900202, 900222],
 [137,
  209103,
  631633,
  631795,
  900135,
  900142,
  900167,
  900212,
  900240,
  900245,
  900265,
  22009977,
  100700804],
 [202101, 213402, 900185, 22010031],
 [631368, 631763, 631765, 900164, 900276, 100700820],
 [213402, 900185, 22010031, 100700864],
 [900185, 900265, 100700824],
 [202901, 203902, 900164, 900217],
 [175, 202901, 900191, 100700864]]

In [151]:
triple_frequent_cameras_list = pathes_rdd_e2.flatMap(lambda x: [(i, j, k) for i in x for j in x for k in x if i<j and j<k])
triple_frequent_cameras_list.take(10)

[(230204, 900107, 900276),
 (230204, 900107, 100700820),
 (230204, 900276, 100700820),
 (900107, 900191, 100700853),
 (900107, 900276, 100700820),
 (900107, 900276, 100700853),
 (900167, 900246, 100700820),
 (900203, 900246, 900269),
 (900203, 900246, 100700820),
 (900226, 900246, 900269)]

In [152]:
triple_frequent_cameras_count = triple_frequent_cameras_list.map(lambda x: (x,1)) \
    .reduceByKey(lambda x, y: x+y)
triple_frequent_cameras_count.take(10)

[((230204, 900107, 900276), 2762),
 ((230204, 900107, 100700820), 895),
 ((900203, 900246, 100700820), 156),
 ((631829, 900202, 900222), 341),
 ((119, 900266, 100700804), 879),
 ((135, 900266, 100700804), 183),
 ((900216, 900240, 900265), 385),
 ((900216, 900265, 900266), 623),
 ((900240, 900256, 900265), 247),
 ((900240, 900265, 900266), 509)]

In [156]:
frequent_triple_camera_pairs_count = triple_frequent_cameras_count.filter(lambda x: x[1]>= support_threshold)
frequent_triple_camera_pairs_count.take(10)

[((230204, 900107, 900276), 2762),
 ((900102, 900142, 100700853), 1389),
 ((205802, 900215, 900234), 2014),
 ((205802, 900234, 100700845), 1007),
 ((212802, 900215, 900234), 1501),
 ((212802, 900234, 100700845), 1083),
 ((900212, 900227, 900244), 1111),
 ((900212, 900244, 22009977), 2312),
 ((900212, 900244, 100700841), 3680),
 ((631829, 900226, 900246), 1221)]

In [157]:
frequent_triple_camera_pairs = frequent_triple_camera_pairs_count.map(lambda x: x[0])
frequent_triple_camera_pairs_list = frequent_triple_camera_pairs.collect()
len(frequent_triple_camera_pairs_list)

438

In [161]:
frequent_triple_camera_pairs.take(10)

[(230204, 900107, 900276),
 (900102, 900142, 100700853),
 (205802, 900215, 900234),
 (205802, 900234, 100700845),
 (212802, 900215, 900234),
 (212802, 900234, 100700845),
 (900212, 900227, 900244),
 (900212, 900244, 22009977),
 (900212, 900244, 100700841),
 (631829, 900226, 900246)]

## k = 4

In [197]:
pathes_rdd_e3 = pathes_rdd_e2.map(lambda x: sorted(set(flatten([ss for ss in subset(x, 3) if ss in frequent_triple_camera_pairs_list])))) \
        .filter(lambda x: len(x) > 3)     
pathes_rdd_e3.take(10)

[[230204, 900107, 900276, 100700820],
 [209103, 631633, 900142, 900167, 900212, 900265, 100700804],
 [631368, 631765, 900164, 900276, 100700820],
 [119, 135, 900216, 900265, 900266, 100700804],
 [117, 900135, 900233, 900234, 900240],
 [212802,
  631829,
  900102,
  900182,
  900223,
  900234,
  900240,
  900246,
  900256,
  22010118,
  100701100,
  100701130],
 [135, 175, 900191, 900216, 900265, 100700804],
 [900102, 900142, 900212, 100700853],
 [631368, 900107, 900164, 100700824],
 [631357, 631363, 631829, 900246]]

In [199]:
quad_frequent_cameras_list = pathes_rdd_e3.flatMap(lambda x: [(i, j, k, m) for i in x for j in x for k in x for m in x if i<j and j<k and k<m])
quad_frequent_cameras_list.take(10)

[(631368, 631765, 900164, 100700820),
 (631765, 900164, 900276, 100700820),
 (135, 900216, 900265, 100700804),
 (212802, 900234, 900256, 22010118),
 (135, 900216, 900265, 100700804),
 (900102, 900142, 900212, 100700853),
 (900102, 900212, 900244, 100700853),
 (900139, 900212, 900244, 100700853),
 (900142, 900212, 900244, 100700853),
 (205802, 212802, 900234, 22010118)]

In [200]:
quad_frequent_cameras_count = quad_frequent_cameras_list.map(lambda x: (x,1)) \
    .reduceByKey(lambda x, y: x+y)
quad_frequent_cameras_count.take(10)

[((900139, 900212, 900244, 100700853), 923),
 ((205802, 212802, 900234, 22010118), 1026),
 ((900139, 900212, 900268, 100700853), 364),
 ((900108, 900259, 900268, 22010119), 859),
 ((900102, 900142, 900212, 900244), 1928),
 ((900101, 900212, 900249, 900268), 346),
 ((631765, 900107, 900276, 100700820), 697),
 ((22010087, 22010088, 22010094, 22010095), 2098),
 ((900212, 900225, 900244, 900269), 757),
 ((900212, 900244, 900273, 100700839), 702)]

In [201]:
frequent_quad_camera_pairs_count = quad_frequent_cameras_count.filter(lambda x: x[1]>= support_threshold)
frequent_quad_camera_pairs_count.take(10)

[((205802, 212802, 900234, 22010118), 1026),
 ((900102, 900142, 900212, 900244), 1928),
 ((22010087, 22010088, 22010094, 22010095), 2098),
 ((900142, 900212, 900273, 100700853), 1116),
 ((900193, 900212, 900244, 100700839), 1543),
 ((900142, 900212, 900249, 100700853), 1387),
 ((900212, 900244, 900249, 100700839), 1023),
 ((900102, 900212, 900244, 100700853), 1243),
 ((900142, 900212, 900244, 100700853), 4949),
 ((142, 900215, 900234, 900256), 1126)]

In [203]:
frequent_quad_camera_pairs = frequent_quad_camera_pairs_count.map(lambda x: x[0])
frequent_quad_camera_pairs_list = frequent_quad_camera_pairs.collect()
len(frequent_quad_camera_pairs_list)

41

## k = 5

In [205]:
pathes_rdd_e4 = pathes_rdd_e3.map(lambda x: sorted(set(flatten([ss for ss in subset(x, 4) if ss in frequent_quad_camera_pairs_list])))) \
        .filter(lambda x: len(x) > 4)     
pathes_rdd_e4.take(10)

[[205802, 212802, 900215, 900234, 22010118],
 [900102, 900142, 900212, 900244, 100701130],
 [900142, 900152, 900212, 900244, 900249],
 [205802, 212802, 900215, 900234, 22010118],
 [142, 205802, 900215, 900234, 900256],
 [205802, 212802, 900215, 900234, 22010118],
 [142, 205802, 900215, 900234, 900256, 22010118],
 [900102, 900142, 900212, 900244, 100700839],
 [900102, 900212, 900244, 100700853, 100701130],
 [900142, 900202, 900212, 900244, 100700841]]

In [207]:
pathes_rdd_e4.count()

10060

In [209]:
quint_frequent_cameras_list = pathes_rdd_e4.flatMap(lambda x: [(i, j, k, m, n) for i in x for j in x for k in x for m in x for n in x if i<j and j<k and k<m and m<n])
quint_frequent_cameras_list.take(10)

[(205802, 212802, 900215, 900234, 22010118),
 (900102, 900142, 900212, 900244, 100701130),
 (900142, 900152, 900212, 900244, 900249),
 (205802, 212802, 900215, 900234, 22010118),
 (142, 205802, 900215, 900234, 900256),
 (205802, 212802, 900215, 900234, 22010118),
 (142, 205802, 900215, 900234, 900256),
 (142, 205802, 900215, 900234, 22010118),
 (142, 205802, 900215, 900256, 22010118),
 (142, 205802, 900234, 900256, 22010118)]

In [213]:
quint_frequent_cameras_count = quint_frequent_cameras_list.map(lambda x: (x,1)) \
    .reduceByKey(lambda x, y: x+y)
quint_frequent_cameras_count.take(10)

[((205802, 212802, 900215, 900234, 22010118), 413),
 ((900142, 900152, 900212, 900244, 900249), 263),
 ((205802, 900215, 900234, 900256, 22010118), 743),
 ((900102, 900212, 900244, 100700853, 100701130), 222),
 ((212802, 900215, 900234, 900256, 22010118), 303),
 ((900193, 900212, 900225, 900244, 100700839), 13),
 ((900193, 900212, 900225, 900268, 100700839), 13),
 ((900193, 900212, 900244, 900259, 100700839), 13),
 ((900193, 900212, 900259, 900268, 900269), 20),
 ((900193, 900244, 900259, 900268, 900269), 20)]

In [215]:
quint_frequent_cameras_count.sortBy(lambda a: -a[1]) \
    .take(10)

[((900142, 900152, 900212, 900244, 100700853), 882),
 ((900142, 900212, 900244, 100700839, 100700853), 843),
 ((900102, 900142, 900212, 900244, 100700853), 820),
 ((205802, 900215, 900234, 900256, 22010118), 743),
 ((142, 900215, 900234, 900256, 22010118), 732),
 ((900142, 900212, 900244, 900249, 100700853), 721),
 ((900142, 900193, 900212, 900244, 100700853), 718),
 ((900142, 900202, 900212, 900244, 100700853), 593),
 ((900193, 900212, 900244, 100700839, 100700853), 545),
 ((142, 205802, 900215, 900234, 900256), 540)]

In [214]:
frequent_quint_camera_pairs_count = quint_frequent_cameras_count.filter(lambda x: x[1]>= support_threshold)
frequent_quint_camera_pairs_count.take(10)

[]

In [216]:
frequent_quint_camera_pairs = frequent_quint_camera_pairs_count.map(lambda x: x[0])
frequent_quint_camera_pairs_list = frequent_quint_camera_pairs.collect()
len(frequent_quint_camera_pairs_list)

0

# SON

In [219]:
def aprioriPass1(rdd, p):
    son_support_threshold = p*support_threshold

    son_frequent_cameras_count = rdd.flatMap(lambda x: x[1]) \
        .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x, y: x + y) \
                .filter(lambda x: x[1] >= son_support_threshold)
    
    son_frequent_cameras = frequent_cameras_count.map(lambda x: (x[0], 1))#.reduceBykey(add)
    son_pathes = rdd.map(lambda x: x[1])
 
    return son_frequent_cameras_count, son_frequent_cameras, son_pathes

In [225]:
# p1, p2, p3 = 0.4, 0.3, 0.3
p1, p2 = 0.5, 0.5
df_rdd1 = dataframe_rdd.sample(False, p1)
df_rdd = dataframe_rdd.subtractByKey(df_rdd1)
df_rdd2 = df_rdd.sample(False, p2)
# df_rdd3 = df_rdd.subtractByKey(df_rdd2)

son1_support_threshold = p1*support_threshold
son2_support_threshold = p2*support_threshold

## k = 1

### Pass 1

In [226]:
son1_frequent_cameras_count, son1_frequent_cameras, son1_pathes = aprioriPass1(df_rdd1, p1)
son2_frequent_cameras_count, son2_frequent_cameras, son2_pathes = aprioriPass1(df_rdd2, p2)
# son3_frequent_cameras_count, son3_frequent_cameras, son3_pathes = aprioriPass1(df_rdd3, p3)

union_candidates = son1_frequent_cameras.union(son2_frequent_cameras)#.union(son3_frequent_cameras)

In [227]:
pass2_union_candidates = sc.broadcast(union_candidates.collect())
pass2_candidates = pass2_union_candidates.value
print("Map: (F,1)   # F: frequent itemset(camera)")
pass2_candidates

Map: (F,1)   # F: frequent itemset(camera)


[(128, 1),
 (631360, 1),
 (900240, 1),
 (631368, 1),
 (100701144, 1),
 (100700864, 1),
 (100700824, 1),
 (100700832, 1),
 (144, 1),
 (631832, 1),
 (900216, 1),
 (900256, 1),
 (900160, 1),
 (22010112, 1),
 (900272, 1),
 (631352, 1),
 (100701096, 1),
 (22010048, 1),
 (900104, 1),
 (900176, 1),
 (900224, 1),
 (232, 1),
 (900208, 1),
 (112, 1),
 (631776, 1),
 (900120, 1),
 (160, 1),
 (900152, 1),
 (22010088, 1),
 (900232, 1),
 (22010072, 1),
 (631784, 1),
 (100700816, 1),
 (168, 1),
 (100700880, 1),
 (22010120, 1),
 (22010040, 1),
 (22010080, 1),
 (900264, 1),
 (900144, 1),
 (631640, 1),
 (104, 1),
 (900184, 1),
 (22009832, 1),
 (22009912, 1),
 (631608, 1),
 (100701264, 1),
 (22010128, 1),
 (631361, 1),
 (205201, 1),
 (137, 1),
 (22009977, 1),
 (900265, 1),
 (631633, 1),
 (113, 1),
 (900185, 1),
 (153, 1),
 (900217, 1),
 (145, 1),
 (208601, 1),
 (202601, 1),
 (203001, 1),
 (206601, 1),
 (900241, 1),
 (103001, 1),
 (100700857, 1),
 (900225, 1),
 (169, 1),
 (900233, 1),
 (212001, 1),
 (10070

In [228]:
len(pass2_candidates)

796

### Pass 2

In [229]:
def aprioriPass2(iterator):
    baskets = [a[1] for a in iterator]
    candidateCount = {}

    for a in baskets:
        for cand in pass2_candidates:
            if cand[0] in a:
                candidateCount.setdefault(cand,0)
                candidateCount[cand] += 1

    for item in sorted(candidateCount.keys()):
        yield (item, candidateCount[item])

son_frequent_cameras_count = dataframe_rdd.mapPartitions(aprioriPass2) \
    .reduceByKey(lambda x, y: x + y) \
        .filter(lambda x: x[1] >= support_threshold) \
            .map(lambda x: (x[0][0], x[1]))
son_frequent_cameras_count.take(10)
# son_frequent_cameras.collect()

[(117, 19164),
 (149, 2710),
 (157, 1806),
 (101301, 17980),
 (107301, 8640),
 (200301, 11546),
 (202101, 12528),
 (202901, 11874),
 (203101, 6406),
 (203301, 2424)]

In [230]:
son_frequent_cameras_count.count()

393

In [231]:
son_frequent_cameras = son_frequent_cameras_count.map(lambda x: x[0])
son_frequent_cameras_list = son_frequent_cameras.collect()
son_frequent_cameras.take(5)

[117, 149, 157, 101301, 107301]

## k = 2

In [232]:
son1_pathes_rdd_e1 = son1_pathes.map(lambda x: sorted(list(set(x)))) \
    .map(lambda x: [p for p in x if p in son_frequent_cameras_list]) \
        .filter(lambda x: len(x) > 1)     
son2_pathes_rdd_e1 = son2_pathes.map(lambda x: sorted(list(set(x)))) \
    .map(lambda x: [p for p in x if p in son_frequent_cameras_list]) \
        .filter(lambda x: len(x) > 1) 
son2_pathes_rdd_e1.take(10)

[[631362, 631757, 900141, 900167, 900203, 900226, 900246, 900269, 100700820],
 [113, 202101, 213402, 900185, 900243, 22010031],
 [202901, 22010117],
 [175, 202901, 900191, 100700864],
 [103001,
  200202,
  202601,
  631357,
  631832,
  900101,
  900191,
  900237,
  900241,
  900253,
  900268],
 [205201, 205202, 900215, 900253, 22009971, 22009977],
 [153, 213402, 900236, 100701143],
 [128, 900160, 900171, 900233],
 [205802, 900234, 900246, 100700862],
 [207101, 900234, 100700804]]

### Pass 1

In [238]:
son1_paired_frequent_cameras = son1_pathes_rdd_e1.flatMap(lambda x: [(i,j) for i in x for j in x if i<j])
son2_paired_frequent_cameras = son2_pathes_rdd_e1.flatMap(lambda x: [(i,j) for i in x for j in x if i<j])

son1_paired_frequent_cameras_count = son1_paired_frequent_cameras.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
son2_paired_frequent_cameras_count = son2_paired_frequent_cameras.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

son1_frequent_camera_paires_count = son1_paired_frequent_cameras_count.filter(lambda x: x[1]>= son1_support_threshold).map(lambda x: (x[0], 1))
son2_frequent_camera_paires_count = son2_paired_frequent_cameras_count.filter(lambda x: x[1]>= son2_support_threshold).map(lambda x: (x[0], 1))

union_paired_candidates = son1_frequent_camera_paires_count.union(son2_frequent_camera_paires_count)

In [240]:
pass2_union_paired_candidates = sc.broadcast(union_paired_candidates.collect())
pass2_paired_candidates = pass2_union_paired_candidates.value
pass2_paired_candidates

[((900107, 900191), 1),
 ((631361, 631829), 1),
 ((900135, 900171), 1),
 ((900142, 900212), 1),
 ((900245, 22009977), 1),
 ((900222, 900228), 1),
 ((900164, 900234), 1),
 ((117, 900233), 1),
 ((900234, 900240), 1),
 ((119, 900235), 1),
 ((900233, 100700845), 1),
 ((631357, 900265), 1),
 ((900246, 100700820), 1),
 ((900107, 900255), 1),
 ((900102, 900212), 1),
 ((101301, 100700841), 1),
 ((631357, 100700841), 1),
 ((900101, 100700841), 1),
 ((900217, 900269), 1),
 ((900234, 900256), 1),
 ((900240, 100700862), 1),
 ((900142, 900244), 1),
 ((114, 232), 1),
 ((631367, 631763), 1),
 ((631833, 900269), 1),
 ((900212, 900234), 1),
 ((900212, 100701130), 1),
 ((900213, 900249), 1),
 ((900225, 900269), 1),
 ((900249, 900269), 1),
 ((900269, 900273), 1),
 ((103002, 900256), 1),
 ((900174, 100700804), 1),
 ((900135, 900235), 1),
 ((900242, 100700864), 1),
 ((114, 631776), 1),
 ((900155, 900207), 1),
 ((900155, 100700839), 1),
 ((900222, 100700868), 1),
 ((900246, 900276), 1),
 ((900246, 100700868

In [241]:
len(pass2_paired_candidates)

2100

### Pass 2

In [242]:
def aprioriPass2_2(iterator):
    baskets = [a[1] for a in iterator]
    candidateCount = {}

    for a in baskets:
        for cand in pass2_paired_candidates:
            if set(cand[0]).issubset(a):
                candidateCount.setdefault(cand,0)
                candidateCount[cand] += 1
    
    for item in sorted(candidateCount.keys()):
        yield (item, candidateCount[item])

son_frequent_paired_cameras = dataframe_rdd.mapPartitions(aprioriPass2_2) \
    .reduceByKey(lambda x, y: x + y) \
        .filter(lambda x: x[1] >= support_threshold) \
            .map(lambda x: (x[0][0], x[1]))
son_frequent_paired_cameras.take(10)

[((112, 900101), 1226),
 ((114, 631795), 1452),
 ((114, 900107), 2146),
 ((114, 900243), 1043),
 ((115, 900236), 1575),
 ((117, 900234), 1625),
 ((117, 900266), 4244),
 ((123, 100700804), 1247),
 ((135, 900216), 10682),
 ((142, 900215), 8198)]

In [243]:
son_frequent_paired_cameras.count()

1525